### ----------------------------------------------------------------------------------------------------------
## 社会変革型 ライフサイエンス・ヘルスケア データサイエンティスト育成講座
# 多変量解析 講義用資料（配布用）
### ----------------------------------------------------------------------------------------------------------

## 2. 統計的解析と仮説検定、検出力 

In [ ]:
# 必要なライブラリのインポート

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
from scipy.stats import norm, t
%matplotlib inline

In [ ]:
# サンプルサイズ計算のための関数定義

def ssize(data,two_sided=True):
    if two_sided:
        z_alpha = norm.ppf(1-data['alpha']/2)
    else:
        z_alpha = norm.ppf(1-data['alpha'])
    z_beta = norm.ppf(1-data['beta'])
    n = np.power(data['sigma']/data['effect'],2)*np.power((z_alpha+z_beta),2)
    return n

In [ ]:
# 練習問題 (仮説検定と検出力)
# 以下のパラメータを定義した上で、関数を使用してください。

test_data = {'effect':,
             'sigma':,
             'alpha':,
             'beta':
             }

## 3-1. 単回帰

In [ ]:
# サンプル生成用関数の定義

def generate_sample(params,nsample=20,xrange=[0,10]):
    rsample = np.zeros([nsample,2])
    for i in range(nsample):
        err = np.random.normal(0,params['sigma'],1)
        x = np.random.uniform(xrange[0],xrange[1],1)
        y = params['b_1'] + params['b_2']*x + err
        rsample[i,:] = y[0],x
        
    return rsample

In [ ]:
# 試しにサンプルを生成してプロット

params = {'b_1':3,
          'b_2':2.5,
          'sigma':10
          }

rsample = generate_sample(params,nsample=50)
plt.scatter(rsample[:,1],rsample[:,0])
plt.xlabel('x')
plt.ylabel('y')
plt.show()

In [ ]:
# 練習問題　（単回帰）


In [ ]:
# linear regression with statsmodels

x = sm.add_constant(rsample[:,1])
mod = sm.OLS(rsample[:,0], x)
res = mod.fit()

print(res.summary())

## 3-2. 重回帰

In [ ]:
# サンプル生成関数の定義

def generate_sample_2d(params,nsample=50,xrange=[[0,10],[0,10]],collinear=False):
    rsample = np.zeros([nsample,3])
    for i in range(nsample):
        err = np.random.normal(0,params['sigma'],1)
        if collinear:
            err_collin = np.random.normal(0,params['sigma_collin'],1)
            x_1 = np.random.uniform(xrange[0][0],xrange[0][1],1)
            x_2 = x_1*params['b_2_collin']+err_collin          
        else:
            x_1 = np.random.uniform(xrange[0][0],xrange[0][1],1)
            x_2 = np.random.uniform(xrange[1][0],xrange[1][1],1)
        y = params['b_1'] + params['b_2']*x_1 + params['b_3']*x_2 + err
        rsample[i,:] = y[0],x_1,x_2
        
    return rsample

In [ ]:
# 試しにプロット

from mpl_toolkits.mplot3d import Axes3D

params = {'b_1': 3,
          'b_2': 2.5,
          'b_3': 1.2,
          'sigma': 1
          }

rsample = generate_sample_2d(params)
fig = plt.figure() 
ax = fig.add_subplot(111, projection='3d')
ax.scatter(rsample[:,1],rsample[:,2],rsample[:,0])
plt.show()

In [ ]:
# statsmodelsを使用してモデルをフィッティングします

params = {'b_1': 3,
          'b_2': 2.5,
          'b_3': 1.2,
          'sigma': 7
          }
rsample = generate_sample_2d(params,nsample=100)

# 定数に相当する列をデータに付加
x = sm.add_constant(rsample[:,1:])

mod = sm.OLS(rsample[:,0], x)
res = mod.fit()

res.summary()

In [ ]:
# 多重共線性ありの場合のサンプル生成

params = {'b_1': 3,
          'b_2': 2.5,
          'b_3': 1.2,
          'sigma': 7,
          'sigma_collin': 1,
          'b_2_collin': 1
          }

fig, ax = plt.subplots(1,2,figsize=[8,4],sharex=True,sharey=True)
rsample_1 = generate_sample_2d(params)
rsample_2 = generate_sample_2d(params,collinear=True)

ax[0].scatter(rsample_1[:,1],rsample_1[:,2])
ax[0].set_xlabel('x_1')
ax[0].set_ylabel('x_2')
ax[0].set_title('without collinearity')
ax[1].scatter(rsample_2[:,1],rsample_2[:,2])
ax[1].set_title('with collinearity')
plt.show()

In [ ]:
# 3d plot

from mpl_toolkits.mplot3d import Axes3D

fig = plt.figure(figsize=[10,4]) 
ax1 = fig.add_subplot(121, projection='3d')
ax2 = fig.add_subplot(122, projection='3d')
ax1.scatter(rsample_1[:,1],rsample_1[:,2],rsample_1[:,0])
ax1.set_title('without collinearity')
ax2.scatter(rsample_2[:,1],rsample_2[:,2],rsample_2[:,0])
ax2.set_title('with collinearity')
plt.show()

In [ ]:
# 多重共線性が認められるデータに対するモデル当てはめ

params = {'b_1': 3,
          'b_2': 2.5,
          'b_3': 1.2,
          'sigma': 7,
          'sigma_collin': 1,
          'b_2_collin': 1
          }

rsample = generate_sample_2d(params,nsample=100,collinear=True)
x = sm.add_constant(rsample[:,1:])
mod = sm.OLS(rsample[:,0], x)
res = mod.fit()

print(res.summary())

#### 練習問題  (重回帰)

In [ ]:
# 練習問題 (重回帰)


In [ ]:
# 分散拡大係数の計算

from statsmodels.stats.outliers_influence import variance_inflation_factor

params['sigma_collin'] = 1
rsample = generate_sample_2d(params,nsample=100,collinear=True)
vif = variance_inflation_factor(rsample[:,1:],0)
print(vif)

## 5. ロジスティック回帰

In [ ]:
# pima indiansデータの読み込み

# Jupyter Notebookのホームディレクトリに下記csvファイルを格納してください
filename = "pima-indians-diabetes.data.csv"

df = pd.read_csv(filename, sep=',',
                 names=[
                     'Pregnancies',
                     'Glucose',
                     'Blood Pressure',
                     'Skin Thickness',
                     'Insulin',
                     'BMI',
                     'Diabetes Pedigree Function',
                     'Age',
                     'Outcome'
                 ]
                )

# 平均値による欠損値の補完 
imputer_cols = df.columns[[1,2,3,4,5,6]]
correct_df = df.copy()

for i in imputer_cols:
    correct_df[i] = correct_df[i].mask(df[i]==0, df[i].mean())

correct_df.head()

In [ ]:
# statsmodels.formulaのインポート

import statsmodels.formula.api as smf

In [ ]:
# 後で使用するバープロット用の関数定義

def plot_category_odds(data):
    odds_ratio = np.zeros([3,1])
    for i in range(3):
        odds_ratio[i] = data[2*i+1]/data[2*i]

    left = np.array([1,2,3])
    labels = [l for l in group_summary.index.levels[0]]
    plt.bar(left,odds_ratio[:,0],tick_label=labels)
    plt.ylabel('odds ratio')
    plt.show()

In [ ]:
# BMIデータの確認

plt.hist(correct_df['BMI'],bins=20,edgecolor='k')
plt.xlabel('BMI')
plt.ylabel('count')
plt.title('BMI data from Pima Indians data')
plt.show()

In [ ]:
# オッズ比の計算

tmp = pd.cut(correct_df['BMI'],[0,30,100])
a = pd.concat([tmp,correct_df['Outcome']],axis=1)
a.groupby(['BMI','Outcome']).size()

In [ ]:
# statsmodelsを使用したロジスティック回帰

# BMIをカテゴリデータ化
data_BMI_cat = pd.cut(correct_df['BMI'],[0,30,100],labels=[0,1])

# Outcomeを結合
data_BMI_cat = pd.concat([data_BMI_cat,correct_df['Outcome']],axis=1)

print(data_BMI_cat.head())

In [ ]:
# モデルのフィッティング

# 説明変数の指定
f = 'Outcome ~ BMI'
logitfit = smf.logit(formula=str(f), data=data_BMI_cat).fit()

print(logitfit.summary())

# パラメータは対数オッズ比を表すため、expの肩に乗せることでオッズ比を算出
print('odds ratio = ' + str(np.exp(logitfit.params[1])))

# 回帰係数のp value
print('p-value = ' + str(logitfit.pvalues[1]))

In [ ]:
# BMIを連続変数のまま、説明変数として使用するモデル

# BMIを説明変数として指定
f = 'Outcome ~ BMI'
logitfit = smf.logit(formula=str(f), data=correct_df).fit()

# オッズ比を算出
print('odds ratio = ' + str(np.exp(logitfit.params[1])))

# 回帰係数のp value
print('p-value = ' + str(logitfit.pvalues[1]))

print(logitfit.summary())

In [ ]:
# BMIを3カテゴリに分けて使用

data_BMI_cat_3 = pd.cut(correct_df['BMI'],[0,25,30,100],labels=['normal','medium','obesity'])
data_BMI_cat_3 = pd.concat([data_BMI_cat_3,correct_df['Outcome']],axis=1)
group_summary = data_BMI_cat_3.groupby(['BMI','Outcome']).size()
print(data_BMI_cat_3.head())

plot_category_odds(group_summary)

f = 'Outcome ~ BMI'
logitfit = smf.logit(formula=str(f), data=data_BMI_cat_3).fit()

print(logitfit.summary())

In [ ]:
# sample generation関数の定義

def generate_sample_logit_2d(params,nsample=50,xrange=[[0,10],[0,10]],collinear=False):
    rsample = np.zeros([nsample,3])
    for i in range(nsample):
        threshold = np.random.rand()
        if collinear:
            err_collin = np.random.normal(0,params['sigma_collin'],1)
            x_2 = np.random.uniform(xrange[1][0],xrange[1][1],1)
            x_1 = x_2*params['b_collin']+err_collin
        else:
            x_1 = np.random.uniform(xrange[0][0],xrange[0][1],1)
            x_2 = np.random.uniform(xrange[1][0],xrange[1][1],1)
            
        t = params['b_1'] + params['b_2']*x_1 + params['b_3']*x_2
        p = 1/(1+np.exp(-t))
        if p>=threshold: y=1 
        else: y=0
                    
        rsample[i,:] = y,x_1,x_2  
    out = pd.DataFrame(rsample)
    out.columns = ['y','x_1','x_2']
    return out

In [ ]:
# 可視化のための関数定義

def plot_scatter(df_pos,df_neg,cols=['x_1','x_2'],dsize=10):
    plt.scatter(df_pos[cols[0]],df_pos[cols[1]],
                c='orange',
                s=dsize,
                label='positive',
               )
    plt.scatter(df_neg[cols[0]],df_neg[cols[1]],
                c='blue',
                s=dsize,
                label='negative',
                alpha=0.3, 
               )

    plt.xlabel(cols[0])
    plt.ylabel(cols[1])
    plt.legend()
    
    
def plot_bar(df_pos,df_neg,cols=['x_1','x_2']):
    fig, ax = plt.subplots(1,len(cols))
    for i,col in enumerate(cols):
        ax[i].bar([1,2],[df_neg[col].mean(),df_pos[col].mean()],tick_label=['neg','pos'])
        ax[i].set_title(col)
    plt.show()
    
    
def plot_scatter_heat(df_pos,df_neg,res,cols=['x_1','x_2'],dsize=10,interaction=False):
    min_x1 = np.min([np.min(df_pos[cols[0]]),np.min(df_neg[cols[0]])])
    max_x1 = np.max([np.max(df_pos[cols[0]]),np.max(df_neg[cols[0]])])
    min_x2 = np.min([np.min(df_pos[cols[1]]),np.min(df_neg[cols[1]])])
    max_x2 = np.max([np.max(df_pos[cols[1]]),np.max(df_neg[cols[1]])])
                     
    x1 = np.linspace(min_x1,max_x1,50)
    x2 = np.linspace(min_x2,max_x2,50)
    
    prediction = np.zeros([50,50])
    for i in range(50):
        for j in range(50):
            if interaction:
                prediction[i,j] = res.predict({'x_1':x1[i],'x_2':x2[j]})[0]
            else:
                prediction[i,j] = res.predict([1,x1[i],x2[j]])[0]
            
    extent = [min_x1, max_x1, min_x2, max_x2]

    fig = plt.figure(figsize=[8,8])
    ax = fig.add_subplot(1,1,1)

    plot_scatter(df_pos,df_neg,dsize=15)

    im = ax.imshow(prediction.T,
                   extent=extent,
                   origin='lower',
                   cmap='pink'
                  )
    ax.set_xlabel('x_1')
    ax.set_ylabel('x_2')
    plt.show()

In [ ]:
# サンプル生成→可視化

params = {'b_1': -3,
          'b_2': 0.01,
          'b_3': 1,
          'b_collin': 1,
          'sigma_collin': 4}

rsample = generate_sample_logit_2d(params,nsample=100,collinear=True)
rsample_pos = rsample[rsample['y']==1]
rsample_neg = rsample[rsample['y']==0]

plot_scatter(rsample_pos,rsample_neg)
plot_bar(rsample_pos,rsample_neg)

In [ ]:
# モデルによるフィッティング

X = sm.add_constant(rsample[['x_1','x_2']])
logit_mod = sm.Logit(rsample['y'], X)
logit_res = logit_mod.fit(disp=0)

print(logit_res.summary())

In [ ]:
# 結果の可視化

plot_scatter_heat(rsample_pos,rsample_neg,logit_res)

#### 練習問題 (ロジスティックモデル)

In [ ]:
# 練習問題 (ロジスティックモデル)


In [ ]:
# 交互作用を仮定しないロジスティックモデルによるフィッティング

f = 'y ~ x_1+x_2'
logitfit = smf.logit(formula = str(f), data = rsample).fit()

print(logitfit.summary())

In [ ]:
# 結果の可視化

plot_scatter_heat(rsample_pos,rsample_neg,logitfit,interaction=True)

In [ ]:
# 交互作用を仮定したロジスティックモデルによるフィッティング

f = 'y ~ x_1*x_2'
logitfit = smf.logit(formula = str(f), data = rsample).fit()

print(logitfit.summary())

In [ ]:
# 結果の可視化

plot_scatter_heat(rsample_pos,rsample_neg,logitfit,interaction=True)

## 6. 生存分析

In [ ]:
# データとライブラリの準備

from lifelines import datasets, KaplanMeierFitter
from lifelines.plotting import plot_lifetimes

gbsg2_data = datasets.load_gbsg2()
gbsg2_data.head()

## German Breast Cancer Study Group 2
contains the observations of 686 women  
http://ugrad.stat.ubc.ca/R/library/ipred/html/GBSG2.html

| column name | variables |
|:---------|:----------|
| horTh | hormonal therapy, a factor at two levels no and yes |
| age | of the patients in years |
| menostat | menopausal status, a factor at two levels pre (premenopausal) and post (postmenopausal) |
| tsize | tumor size (in mm) |
| tgrade | tumor grade, a ordered factor at levels I < II < III |
| pnodes | number of positive nodes |
| progrec | progesterone receptor (in fmol) |
| estrec | estrogen receptor (in fmol) |
| time | recurrence free survival time (in days) | 
| cens | censoring indicator (0- censored, 1- event) | 

In [ ]:
# timeデータの可視化

time = gbsg2_data['time']
event = gbsg2_data['cens']

time_sorted = time.sort_values().values
plot_lifetimes(time_sorted[:50], event_observed=event[:50])
plt.xlabel('time (days)')
plt.show()

In [ ]:
# Kaplan-meier曲線の描画

kmf = KaplanMeierFitter()
kmf.fit(time, event_observed=event)

kmf.plot(ci_show=False)
plt.title('Survival function')
plt.xlabel('time (days)')
plt.show()

In [ ]:
# ホルモン療法の有無で比較

ax = plt.subplot(111)

therapy = (gbsg2_data["horTh"] == "yes")

kmf.fit(time[therapy], event_observed=event[therapy], label="with hormonal therapy")
kmf.plot(ax=ax,ci_show=False)
kmf.fit(time[~therapy], event_observed=event[~therapy], label="No therapy")
kmf.plot(ax=ax,ci_show=False)

plt.ylim(0, 1)
plt.xlabel('time (days)')
plt.title("Survival time in GBSG2 data")
plt.show()

In [ ]:
# log-rank検定

from lifelines.statistics import logrank_test

results = logrank_test(time[therapy], time[~therapy], event[therapy], event[~therapy], alpha=.99)
results.print_summary()

In [ ]:
# cox model

from lifelines import CoxPHFitter

cph = CoxPHFitter()
gbsg2_data_dm = pd.get_dummies(gbsg2_data,columns=['horTh','menostat','tgrade'],drop_first=True)
cph.fit(gbsg2_data_dm, duration_col='time', event_col='cens', show_progress=True)
cph.print_summary()
cph.plot()